In [17]:
import pandas as pd
import numpy as np



In [18]:
df = pd.read_csv('train.csv')
articles = df['Text']

for i, text in enumerate(articles):
    print("article:{}\t{}".format(i+1, text))
    print()

article:1	lifestyle governs mobile choice faster better funkier hardware alone going help phone firms sell handsets research suggests instead phone firms keen get customers pushing technology sake consumers far interested handsets fit lifestyle screen size onboard memory chip inside shows depth study handset maker ericsson historically industry much focus using technology said dr michael bjorn senior advisor mobile media ericsson consumer enterprise lab stop saying technologies change lives said try speak consumers language help see fits told bbc news website study ericsson interviewed 14 000 mobile phone owners ways use phone people habits remain said dr bjorn move activity mobile phone much convenient way one good example diary writing among younger people said diaries always popular mobile phone especially one equipped camera helps keep different form youngsters use text messages also reflects desire chat keep contact friends lets slightly changed way dr bjorn said although consumer

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# Identify Unique words
vectorizer = CountVectorizer()
vectorizer.fit(articles)
print(f'vector vocabulary - {vectorizer.vocabulary_}\n')

vector vocabulary - {'lifestyle': 7222, 'governs': 5566, 'mobile': 7939, 'choice': 2494, 'faster': 4785, 'better': 1620, 'funkier': 5262, 'hardware': 5798, 'alone': 851, 'going': 5526, 'help': 5898, 'phone': 9010, 'firms': 4961, 'sell': 10811, 'handsets': 5771, 'research': 10186, 'suggests': 11791, 'instead': 6485, 'keen': 6877, 'get': 5432, 'customers': 3339, 'pushing': 9665, 'technology': 12090, 'sake': 10573, 'consumers': 3017, 'far': 4769, 'interested': 6528, 'fit': 4969, 'screen': 10721, 'size': 11124, 'onboard': 8515, 'memory': 7751, 'chip': 2491, 'inside': 6460, 'shows': 11024, 'depth': 3615, 'study': 11710, 'handset': 5770, 'maker': 7497, 'ericsson': 4459, 'historically': 5973, 'industry': 6387, 'much': 8075, 'focus': 5047, 'using': 12845, 'said': 10567, 'dr': 4006, 'michael': 7816, 'bjorn': 1684, 'senior': 10831, 'advisor': 701, 'media': 7714, 'consumer': 3016, 'enterprise': 4405, 'lab': 6993, 'stop': 11613, 'saying': 10637, 'technologies': 12089, 'change': 2406, 'lives': 7307

In [27]:
vector = vectorizer.transform(articles)
print(f'features\n {vectorizer.get_feature_names_out()}\n')
# summarize encoded vector
print(f'vector shape: {vector.shape}\n')
number_of_features = vector.shape[1]

print(f'number of features: {number_of_features}\n')
examples = pd.DataFrame(vector.toarray(), columns=vectorizer.get_feature_names_out())
examples.head()

features
 ['00' '000' '000th' ... 'zooms' 'zooropa' 'zorro']

vector shape: (428, 13518)

number of features: 13518



,00,000,000th,001st,0051,007,0100,0130,028,05,...,zola,zombie,zombies,zone,zonealarm,zones,zoom,zooms,zooropa,zorro
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
